In [7]:
import numpy as np

import random

import nltk
from nltk.stem.porter import PorterStemmer	
stemmer = PorterStemmer()

from transformers import BertTokenizer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import warnings 
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline

import joblib


In [8]:
from DZ1_data import GetData

In [9]:
elrian_dialogues, leonard_dialogues = GetData()

In [10]:
def preprocess_dataset(elrian_dialogues, leonard_dialogues, tokenizer):
    inputs = []
    labels = []

    for elrian_dialogue, leonard_dialogue in zip(elrian_dialogues, leonard_dialogues):
        # Tokenize вопрос и ответ
        elrian_tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(elrian_dialogue, add_special_tokens=True)))
        leonard_tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(leonard_dialogue, add_special_tokens=True)))

        # Формирование входных и выходных данных
        inputs.append(elrian_tokens)
        labels.append(leonard_tokens)

    # Преобразование в тензоры PyTorch
    inputs = [torch.tensor(tokenizer.encode(tokens, add_special_tokens=True)) for tokens in inputs]
    labels = [torch.tensor(tokenizer.encode(tokens, add_special_tokens=True)) for tokens in labels]
    return inputs, labels

In [13]:
# Загрузка предварительно обученного токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Подготовка данных
inputs, labels = preprocess_dataset(elrian_dialogues, leonard_dialogues, tokenizer)


In [14]:
# Вывод примера данных
print("Input:", inputs[0])
print("Label:", labels[0])

Input: tensor([ 101,  101, 2339, 2106, 2017, 2272, 2000, 1996, 3224, 1029,  102,  102])
Label: tensor([ 101,  101, 1045, 1005, 1049, 2559, 2005, 2019, 3418, 3379, 1010, 1045,
        1005, 2310, 2657, 9489, 2055, 2009, 1012,  102,  102])


In [21]:
vectorizer, retrieval_model = train_retrieval_model(inputs, labels)

In [23]:
joblib.dump(vectorizer, "./DZ1model/TFIDF/vectorizer.joblib")
joblib.dump(retrieval_model, './DZ1model/TFIDF/modelTFIDF.joblib')
joblib.dump(labels, "./DZ1model/TFIDF/labels.joblib")
joblib.dump(tokenizer, "./DZ1model/TFIDF/tokenizer.joblib")

['./DZ1model/TFIDF/tokenizer.joblib']